In [1]:

# ===== Week 1 – Consolidate Business Reports (Sections 1–5) WITH INLINE PLOTS =====
from pathlib import Path
from datetime import datetime
import re

BASE_PROJECT_NAME = "SDS-CP036-powercast"
WEEK_TAG = "Wk01"
SECTIONS = [1,2,3,4,5]

# --- Locate a sensible project root (no hardcoded folder names) ---
def find_project_root(start: Path) -> Path:
    p = start.resolve()
    for _ in range(12):
        if (p / "results").exists() or (p / "data").exists():
            return p
        if p.parent == p: break
        p = p.parent
    return start.resolve()

BASE_DIR = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd().resolve()
PROJECT_ROOT = find_project_root(BASE_DIR)

def section_dirs(sec: int):
    sec_tag = f"{WEEK_TAG}_Section{sec}"
    res_dir = PROJECT_ROOT / "results" / sec_tag
    return res_dir, res_dir / "reports", res_dir / "plots"

def expected_report_name(sec: int) -> str:
    return f"{BASE_PROJECT_NAME}_{WEEK_TAG}_Section{sec}_Report_Business.md"

def find_report_md(sec: int):
    res_dir, rep_dir, _ = section_dirs(sec)
    cand = rep_dir / expected_report_name(sec)
    if cand.exists():
        return cand
    if rep_dir.exists():
        any_md = sorted(rep_dir.glob("*Report_Business.md"))
        if any_md: return any_md[0]
    if res_dir.exists():
        any_md = sorted(res_dir.rglob("*Report_Business.md"))
        if any_md: return any_md[0]
    return None

def list_plot_files_for_section(sec: int):
    """Return a best-effort ordered list of plot paths to include for this section.
       1) Respect order of any backticked `plots/<file>` mentions in the MD
       2) Append the remaining PNGs from the plots folder
    """
    res_dir, rep_dir, plots_dir = section_dirs(sec)
    md = find_report_md(sec)
    ordered = []
    seen = set()
    if md and md.exists():
        text = md.read_text(encoding="utf-8")
        # backticked: `plots/xyz.png`
        for m in re.finditer(r"`plots/([^`]+)`", text):
            rel = m.group(1).strip()
            p = plots_dir / rel
            if p.exists() and p.suffix.lower() in {".png", ".jpg", ".jpeg", ".webp"}:
                if p not in seen:
                    ordered.append(p); seen.add(p)
        # images: ![](plots/xyz.png)
        for m in re.finditer(r"!\[[^\]]*\]\(plots/([^)]+)\)", text):
            rel = m.group(1).strip()
            p = plots_dir / rel
            if p.exists() and p.suffix.lower() in {".png", ".jpg", ".jpeg", ".webp"}:
                if p not in seen:
                    ordered.append(p); seen.add(p)
    # Append remaining PNGs in directory
    if plots_dir.exists():
        for p in sorted(plots_dir.glob("*")):
            if p.suffix.lower() in {".png", ".jpg", ".jpeg", ".webp"} and p not in seen:
                ordered.append(p); seen.add(p)
    return ordered

# --- Consolidation ---
CONSOL_DIR = PROJECT_ROOT / "results" / f"{WEEK_TAG}_CONSOLIDATED"
CONSOL_DIR.mkdir(parents=True, exist_ok=True)
OUT_MD = CONSOL_DIR / f"{BASE_PROJECT_NAME}_{WEEK_TAG}_Business_Consolidated.md"

def make_toc(items):
    lines = ["## Table of Contents"]
    for sec, path, title in items:
        lines.append(f"- [Section {sec} — {title}](#section-{sec})")
    return "\n".join(lines) + "\n\n"

def extract_title(md_text: str, default: str):
    m = re.search(r"^#\s+(.+)$", md_text, flags=re.MULTILINE)
    return m.group(1).strip() if m else default

found = []
missing = []
for sec in SECTIONS:
    p = find_report_md(sec)
    if p is None:
        missing.append(sec); continue
    txt = p.read_text(encoding="utf-8")
    title = extract_title(txt, f"{WEEK_TAG} Section {sec}")
    found.append((sec, p, title))

header = f"""# {BASE_PROJECT_NAME} — {WEEK_TAG} Consolidated Business Report (Inline Plots)

Generated on: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
Project root: `{PROJECT_ROOT}`

Includes Sections: {', '.join(str(s) for s,_,_ in found) if found else 'None'}
{"Missing Sections: " + ", ".join(map(str, missing)) if missing else ""}

"""

toc = make_toc(found)

body_parts = []
for sec, path, title in found:
    res_dir, rep_dir, plots_dir = section_dirs(sec)
    section_anchor = f"## Section {sec} — {title}\n"
    txt = path.read_text(encoding="utf-8")
    # Strip the first H1
    txt_no_h1 = re.sub(r"^#\s+.*\n", "", txt, count=1, flags=re.MULTILINE)
    # Normalize backticked plot paths to project-relative paths for readability
    txt_no_h1 = re.sub(r"`plots/([^`]+)`",
                       lambda m: f"`results/{WEEK_TAG}_Section{sec}/plots/{m.group(1)}`",
                       txt_no_h1)
    # Normalize image links similarly
    txt_no_h1 = re.sub(r"!\[([^\]]*)\]\(plots/([^)]+)\)",
                       lambda m: f"![]({('results/' + f'{WEEK_TAG}_Section{sec}/plots/' + m.group(2))})",
                       txt_no_h1)
    # Build inline visuals block
    plot_files = list_plot_files_for_section(sec)
    visuals_block = ""
    if plot_files:
        visuals_block_lines = ["\n### Visuals\n"]
        for p in plot_files:
            rel = p.relative_to(PROJECT_ROOT).as_posix()
            alt = f"Section {sec} — {p.name}"
            visuals_block_lines.append(f"![{alt}]({rel})")
        visuals_block = "\n".join(visuals_block_lines) + "\n"
    body_parts.append(section_anchor + "\n" + txt_no_h1.strip() + "\n" + visuals_block)

OUT_MD.write_text(header + toc + "\n".join(body_parts), encoding="utf-8")

print("✅ Consolidation complete (inline plots).")
print("Output:", OUT_MD)
print("Found sections:", [s for s,_,_ in found])
print("Missing sections:", missing)


✅ Consolidation complete (inline plots).
Output: /home/6376f5a9-d12b-4255-9426-c0091ad440a7/Powercast/results/Wk01_CONSOLIDATED/SDS-CP036-powercast_Wk01_Business_Consolidated.md
Found sections: [1, 2, 3, 4, 5]
Missing sections: []
